In [2]:
#O que eu preciso saber nesse relatório:
# 1. Quantas vezes empresas ganharam licitações de merenda em quais municípios?
#    LICITAÇÂO >> LOTES >> CONCORRENTES

# 2. Qual é a idade da empresa que ganhou as licitações (ver se é uma que foi criada pra isso)
# 3. Cruzar empresas vencedoras com a RAIS
# 4. Olhar se essas licitações possuem aditivos de licitação (sinal de abuso)
# 5. Ver tudo isso por municpipio, de forma que possamos buscar por um município
# 6. Ver se consigo dizer quanto custou cada produto

In [78]:
#1.a Licitações
import pandas as pd
pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_columns', 0) # Display any number of columns
pd.set_option('display.max_rows', 0) # Display any number of rows

#os.chdir("C:\Users\coliv\licitacoes_merenda_rs\dbs\2017_licitacoes") 
# outro jeito de setwd()
lic_2017 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2017_licitacoes\licitacao.csv', delimiter =',', encoding='utf8')
lic_2018 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2018_licitacoes\licitacao.csv', delimiter =',', encoding='utf8')
lic_2019 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2019_licitacoes\licitacao.csv', delimiter =',', encoding='utf8')

# lic_2017.info() 62440

#modalidade igual a merenda ("CPP")
lic_2017 = lic_2017[lic_2017['CD_TIPO_MODALIDADE'] == 'CPP']
lic_2018 = lic_2018[lic_2018['CD_TIPO_MODALIDADE'] == 'CPP']
lic_2019 = lic_2019[lic_2019['CD_TIPO_MODALIDADE'] == 'CPP']

# juntando tudo em um arquivo só
lic_merenda_rs = pd.concat([lic_2017, lic_2018, lic_2019])

# Só as licitações que já foram concluídas:
lic_merenda_rs = lic_merenda_rs[lic_merenda_rs['CD_TIPO_FASE_ATUAL'] == 'ADH']

#A chave deste registro é: NR_LICITACAO, ANO_LICITACAO e CD_TIPO_MODALIDADE
lic_merenda_rs = lic_merenda_rs[['CNPJ_ORGAO_GERENCIADOR', 'NM_ORGAO_GERENCIADOR', 'NR_LICITACAO', 'ANO_LICITACAO' , 'CD_TIPO_MODALIDADE', 'CD_ORGAO', 'NM_ORGAO', 'TP_CARACTERISTICA_OBJETO', 'TP_NATUREZA', 'DS_OBJETO', 'VL_LICITACAO', 'VL_HOMOLOGADO', 'PC_TX_ESTIMADA', 'PC_TX_HOMOLOGADA', 'CD_TIPO_FASE_ATUAL']]

# Algumas das colunas estão como interger e eu quero como string (object):
lic_merenda_rs[['CNPJ_ORGAO_GERENCIADOR', 'NR_LICITACAO', 'ANO_LICITACAO']] = lic_merenda_rs[['CNPJ_ORGAO_GERENCIADOR', 'NR_LICITACAO', 'ANO_LICITACAO']].astype(str)

lic_merenda_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 953 entries, 4036 to 34966
Data columns (total 15 columns):
CNPJ_ORGAO_GERENCIADOR      953 non-null object
NM_ORGAO_GERENCIADOR        10 non-null object
NR_LICITACAO                953 non-null object
ANO_LICITACAO               953 non-null object
CD_TIPO_MODALIDADE          953 non-null object
CD_ORGAO                    953 non-null int64
NM_ORGAO                    953 non-null object
TP_CARACTERISTICA_OBJETO    953 non-null object
TP_NATUREZA                 953 non-null object
DS_OBJETO                   953 non-null object
VL_LICITACAO                953 non-null float64
VL_HOMOLOGADO               118 non-null object
PC_TX_ESTIMADA              0 non-null float64
PC_TX_HOMOLOGADA            0 non-null float64
CD_TIPO_FASE_ATUAL          953 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 119.1+ KB


In [44]:
# 1.b informações sobre quem concorreu

# Abrindo arquivos de concorrentes , com as infos das empresas
concorrentes_2017 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2017_licitacoes\licitante.csv', delimiter =',', encoding='utf8')
concorrentes_2018 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2018_licitacoes\licitante.csv', delimiter =',', encoding='utf8')
concorrentes_2019 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2019_licitacoes\licitante.csv', delimiter =',', encoding='utf8')

#Juntando todos os concorrentes de todos os anos:
concorrentes_rs = pd.concat([concorrentes_2017, concorrentes_2018, concorrentes_2019])

#Dropando colunas que não me ajudam (do representante da empresa)
concorrentes_rs = concorrentes_rs.drop(['TP_DOCUMENTO.1', 'NR_DOCUMENTO.1', 'TP_CONDICAO'], axis = 1)

concorrentes_rs[['CD_ORGAO', 'NR_LICITACAO', 'ANO_LICITACAO']]= concorrentes_rs[['CD_ORGAO', 'NR_LICITACAO', 'ANO_LICITACAO']].astype(str)

c:\users\coliv\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\coliv\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
#1.c informações sobre as ofertas

# Abrindo os arquivos de lotes (onde eu consigo ver quem vanhou a licitação)
lote_2017 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2017_licitacoes\lote.csv', delimiter =',', encoding='utf8')
lote_2018 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2018_licitacoes\lote.csv', delimiter =',', encoding='utf8')
lote_2019 = pd.read_csv(r'C:\Users\coliv\licitacoes_merenda_rs\dbs\2019_licitacoes\lote.csv', delimiter =',', encoding='utf8')

lote_2017[['CD_ORGAO', 'NR_LICITACAO', 'ANO_LICITACAO']] = lote_2017[['CD_ORGAO', 'NR_LICITACAO', 'ANO_LICITACAO']].astype(str)
lote_2018[['CD_ORGAO', 'NR_LICITACAO', 'ANO_LICITACAO']] = lote_2018[['CD_ORGAO', 'NR_LICITACAO', 'ANO_LICITACAO']].astype(str)
lote_2019[['CD_ORGAO', 'NR_LICITACAO', 'ANO_LICITACAO']] = lote_2019[['CD_ORGAO', 'NR_LICITACAO', 'ANO_LICITACAO']].astype(str)

lotes = pd.concat([lote_2017, lote_2018, lote_2019])

#mudando uma coluna só

lotes.NR_LOTE = lotes.NR_LOTE.astype(str)
# A chave deste registro é: NR_LICITACAO, ANO_LICITACAO, CD_TIPO_MODALIDADE e NR_LOTE

In [67]:
#Juntando licitações e lotes

licitacoes_lotes = pd.merge(lic_merenda_rs, lotes, how = "left", on = ['NR_LICITACAO', 'ANO_LICITACAO', 'CD_TIPO_MODALIDADE'])

In [84]:
# Selecionando apenas os vencedores
#lotes = lotes[lotes.TP_RESULTADO_LOTE == 'A']

licitacoes_lotes.groupby(by=['CNPJ_ORGAO_GERENCIADOR'])['CNPJ_ORGAO_GERENCIADOR'].count()


CNPJ_ORGAO_GERENCIADOR
1601857000120.0         25
87246120000151.0        58
87613154000137.0        71
88372883000101.0        84
88546890000182.0       106
94309291000148.0        53
nan                 167607
Name: CNPJ_ORGAO_GERENCIADOR, dtype: int64

In [96]:

# print(licitacoes_lotes.groupby(by=['NR_LICITACAO', 'ANO_LICITACAO' , 'CD_TIPO_MODALIDADE']).count().sum())

print(licitacoes_lotes.groupby(['CNPJ_ORGAO_GERENCIADOR','ANO_LICITACAO' , 'NR_LICITACAO', 'CD_TIPO_MODALIDADE'])['NR_LOTE'].agg(['count']))

                                                                      count
CNPJ_ORGAO_GERENCIADOR ANO_LICITACAO NR_LICITACAO CD_TIPO_MODALIDADE       
1601857000120.0        2018          13.0         CPP                    25
87246120000151.0       2019          33.0         CPP                    58
87613154000137.0       2018          6.0          CPP                    36
                       2019          3.0          CPP                    35
88372883000101.0       2019          5.0          CPP                    84
88546890000182.0       2018          106.0        CPP                    22
                       2019          5.0          CPP                    84
94309291000148.0       2018          16.0         CPP                     7
                                     17.0         CPP                     6
                                     18.0         CPP                    40
nan                    2017          1.0          CPP                  2385
            